# aside_EDA_exploration_02overAll.IPyNb
- event_contents 디테일 탐색하기
- event_contents 를 종류별로 쪼개서 탐색하기

### 1.0 gdrive 마운트 / Import + 함수정의
- 코드를 사용해서 모듈 import 로 마운트 하거나
- 코랩 폴더에서 드라이브 마운트 클릭 하거나

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# 데이터 전처리를 병렬처리로 시도해 봄!
- 데이터 전처리에 메모리도 많이 먹고, 시간도 오래걸림
- 병렬처리하면 5배 이상 빨라진다니 한번 시도해 봄

In [2]:
import multiprocessing as mp

from parse import *
from multiprocessing import Pool

num_cores = mp.cpu_count()               # cpu의 코어 수를 반환
print(f"CPU CORES = {num_cores} CORES!")        # CPU CORES = 16 CORES!

CPU CORES = 16 CORES!


In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

In [4]:
# 데이터가 많아 5분 정도 걸립니다.
dir_base = '/home/yk/0325_Starcraft/competition/'

raw      = 'c03_starcraft_prediction/data_raw/'
remake   = 'c03_starcraft_prediction/data_remake/'
submit   = 'c03_starcraft_prediction/data_submit/'

assets = '/home/yk/0325_Starcraft/competition/_assets'

### 사용함수의 정의

In [5]:
os.chdir(dir_base)
from _assets.modules import *


# OS 화일 및 DF 정보조회를 위한 탐색 모듈



## 2.0 Train Data Set 불러오기

| 컬럼명  | 값    | type     | 설명                  | 비고   |
|:-------:|:------|----------|-----------------------|--------|
| game_id |33781  |int       |경기 구분 기호         | unique (multiple) 
| winner  |0.6523 |float     |player-1 승리확률      | 0.6287
| time    |2.24   |float     |경기시간 (60분법 구분) | 2분24초
| player  |0      |int       |player 0 - 첫번째 선수 
|         |1      |int       |player 1 - 두번째 선수          
| species |T      |Object    |테란 (Terran)    |     
|         |P      |Object    |프로토스 (Protos)|      
|         |Z      |Object    |저그 (Zerg)      |      
| event   |Ability           |Object |생산, 공격 등 선수의 주요 행동
|         |AddToControlGroup |Object |부대에 추가
|         |Camera            |Object |시점 선택
|         |ControlGroup      |Object |부대 행동
|         |GetControlGroup   |Object |부대 불러오기
|         |Right             |Object |Click : 마우스 우클릭
|         |Selection         |Object |객체 선택
|         |SetControlGroup   |Object |부대 지정
|event_contents|@(좌표, 등)  | object|이벤트에 관한 상세설명 


 

In [6]:
%%time
df_train = pd.read_csv(dir_base + raw + 'train.csv')

CPU times: user 46 s, sys: 4.24 s, total: 50.2 s
Wall time: 54.5 s


In [7]:
show_infoDF_from(df_train)

*** DATA SHAPE = [ 67,091,776 x 7 ]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67091776 entries, 0 to 67091775
Data columns (total 7 columns):
game_id           int64
winner            int64
time              float64
player            int64
species           object
event             object
event_contents    object
dtypes: float64(1), int64(3), object(3)
memory usage: 3.5+ GB
None

            game_id        winner          time        player
count  6.709178e+07  6.709178e+07  6.709178e+07  6.709178e+07
mean   1.944981e+04  5.012076e-01  4.277912e+00  4.992699e-01
std    1.122980e+04  4.999985e-01  2.709741e+00  4.999995e-01
min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
25%    9.718000e+03  0.000000e+00  2.120000e+00  0.000000e+00
50%    1.948500e+04  1.000000e+00  4.150000e+00  0.000000e+00
75%    2.919700e+04  1.000000e+00  6.360000e+00  1.000000e+00
max    3.887100e+04  1.000000e+00  1.059000e+01  1.000000e+00



In [8]:
df_train.shape         # (67091776, 7)
df_train.head(5)

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"


### 이벤트 Detail 탐색을 위해 각 이벤트 딕트를 만든다

In [9]:
%%time
# CPU times: user 1min 44s, sys: 1.28 s, total: 1min 45s
# Wall time: 1min 45s

# for item in ['P0_species','P1_species']:
#     print(df_train_refine[item].value_counts(True), end='\n\n')

counts = {
        'Camera'            : df_train['event_contents'][df_train['event'] == 'Camera'].value_counts(),
        'Right_Click'       : df_train['event_contents'][df_train['event'] == 'Right Click'].value_counts(),
        'GetControlGroup'   : df_train['event_contents'][df_train['event'] == 'GetControlGroup'].value_counts(),
        'Selection'         : df_train['event_contents'][df_train['event'] == 'Selection'].value_counts(),
        'Ability'           : df_train['event_contents'][df_train['event'] == 'Ability'].value_counts(),
        'SetControlGroup'   : df_train['event_contents'][df_train['event'] == 'SetControlGroup'].value_counts(),
        'AddToControlGroup' : df_train['event_contents'][df_train['event'] == 'AddToControlGroup'].value_counts(),
        'ControlGroup'      : df_train['event_contents'][df_train['event'] == 'ControlGroup'].value_counts(),
    }


CPU times: user 1min 17s, sys: 1.58 s, total: 1min 19s
Wall time: 1min 2s


In [10]:
counts['Camera']

at (143.5, 22.7578125)             89727
at (24.5, 145.7578125)             88246
at (0.0, 0.0)                      82567
at (33.5, 23.7578125)              73001
at (26.5, 126.76171875)            71466
                                   ...  
at (92.06640625, 77.49609375)          1
at (147.56640625, 113.14453125)        1
at (114.10546875, 51.72265625)         1
at (41.91015625, 43.37890625)          1
at (30.3046875, 53.171875)             1
Name: event_contents, Length: 23902568, dtype: int64

In [11]:
counts['Right_Click']

Target: None [00A80001]; Location: (18.0, 147.5, 49120)                              27042
Target: None [00040001]; Location: (29.0, 15.5, 49104)                               25418
Target: DestructibleSignsConstruction [00740001]; Location: (20.0, 128.5, 49136)     24670
Target: DestructibleSignsConstruction [02580001]; Location: (157.0, 142.5, 49120)    22346
Target: DestructibleSignsConstruction [00280001]; Location: (163.0, 15.5, 49088)     21714
                                                                                     ...  
Location: (117.609375, 103.53466796875, 32736)                                           1
Location: (61.6357421875, 129.616943359375, 40941)                                       1
Location: (132.668701171875, 139.484130859375, 45871)                                    1
Location: (34.922119140625, 145.04052734375, 49127)                                      1
Location: (35.598876953125, 137.25048828125, 49127)                                      1

## rightClick(DF)를 세미콜론 기준, 스플릿하는 멀티프로세싱
- 새로 컬럼을 만든다.

In [12]:
def get_basic_df(dict, key_name='Right_Click'):
    """counts 딕트에서 기본 key df 를 만들고 컬럼 제정렬 한다."""
    df = pd.DataFrame(dict[key_name])
    df['details'] = df.index.copy()
    df.index = np.arange(len(df['details']))
    df = df[['details', 'event_contents']]              # re-order column position 

    # df.columns = ['details', 'counts']                # rename ... not shows result
    df.rename(columns={'event_contents' : 'counts'})    # rename ... shows result [추천]
    return df

In [13]:
def parallelize_dataframe(df, func):
    """DF 를 코어수만큼 쪼개서, 매핑하고 다시 연결시킴
    Pandas DataFrame을 병렬처리 하는 방법 = https://bit.ly/2JKTw5v
    """
    num_cores = mp.cpu_count()               # cpu의 코어 수를 반환
    print(f"CPU CORES = {num_cores} CORES!")
    
    df_split = np.array_split(df, num_cores)
    pool = Pool(num_cores)

    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [14]:
# 전처리 필요합수를 재지정 한다.
def func_helper(df):
    """HELPER() for multi-process = 처리함수를 별도수정 필요"""
    df['num_split'] = df['details'].apply(lambda x: len(x.split(';')))
    df['splits_01'] = df['details'].apply(lambda x: x.split(';')) 
    df['splits_02'] = df['splits_01'].apply(lambda x: x[0].split(':')[1]) 
    return df

In [15]:
df_rightClick = get_basic_df(dict=counts, key_name='Right_Click')
df_rightClick.head()

,details,event_contents
0,"Target: None [00A80001]; Location: (18.0, 147....",27042
1,"Target: None [00040001]; Location: (29.0, 15.5...",25418
2,Target: DestructibleSignsConstruction [0074000...,24670
3,Target: DestructibleSignsConstruction [0258000...,22346
4,Target: DestructibleSignsConstruction [0028000...,21714


In [16]:
%%time
df_multi = parallelize_dataframe(df_rightClick, func_helper)

CPU CORES = 16 CORES!
CPU times: user 37.6 s, sys: 5.63 s, total: 43.2 s
Wall time: 21.9 s


In [17]:
# df_multi.sort_values(by='num_split', ascending=False).head(10)
cols = ['event_contents', 'num_split', 'splits_01', 'splits_02', ]
df_multi[cols].sort_values(by='event_contents', ascending=False).head(50)

,event_contents,num_split,splits_01,splits_02
0,27042,2,"[Target: None [00A80001], Location: (18.0, 14...",None [00A80001]
1,25418,2,"[Target: None [00040001], Location: (29.0, 15...",None [00040001]
2,24670,2,[Target: DestructibleSignsConstruction [007400...,DestructibleSignsConstruction [00740001]
3,22346,2,[Target: DestructibleSignsConstruction [025800...,DestructibleSignsConstruction [02580001]
4,21714,2,[Target: DestructibleSignsConstruction [002800...,DestructibleSignsConstruction [00280001]
5,20637,2,[Target: DestructibleSignsConstruction [026000...,DestructibleSignsConstruction [02600001]
6,20622,2,[Target: DestructibleSignsConstruction [000C00...,DestructibleSignsConstruction [000C0001]
7,20200,2,"[Target: None [016C0001], Location: (150.0, 2...",None [016C0001]
8,19628,2,[Target: DestructibleSignsConstruction [003800...,DestructibleSignsConstruction [00380001]
9,19574,2,[Target: DestructibleSignsConstruction [008000...,DestructibleSignsConstruction [00800001]


In [18]:
df_Ability = get_basic_df(dict=counts, key_name='Ability')
df_Ability.head(10)

,details,event_contents
0,(1360) - TrainSCV,350376
1,(1820) - MorphDrone,287254
2,(15E0) - TrainProbe,258765
3,(13E0) - TrainMarine,134801
4,(1822) - MorphOverlord,90700
5,(1821) - MorphZergling,83416
6,(1E60) - TrainQueen,51130
7,(13A0) - LowerSupplyDepot,42761
8,(1C60) - UpgradeToOrbitalCommand,34531
9,(480) - Stop,30424


In [19]:
# 전처리 필요합수를 재지정 한다.
def func_helper(df):
    """HELPER() for multi-process = 처리함수를 별도수정 필요"""
    df['num_split'] = df['details'].apply(lambda x: len(x.split('-')))
    df['splits_01'] = df['details'].apply(lambda x: x.split('-')) 
    df['splits_02'] = df['splits_01'].apply(lambda x: x[0])
    df['splits_03'] = df['splits_01'].apply(lambda x: x[-1])  
    return df

In [20]:
%%time
df_multi_Ability = parallelize_dataframe(df_Ability, func_helper)

CPU CORES = 16 CORES!
CPU times: user 3.96 s, sys: 2.26 s, total: 6.22 s
Wall time: 4.81 s


In [21]:
# df_multi.sort_values(by='num_split', ascending=False).head(10)
cols = ['event_contents', 'num_split', 'splits_01', 'splits_02', 'splits_03',]
df_multi_Ability[cols].sort_values(by='event_contents', ascending=False).head(50)

,event_contents,num_split,splits_01,splits_02,splits_03
0,350376,2,"[(1360) , TrainSCV]",(1360),TrainSCV
1,287254,2,"[(1820) , MorphDrone]",(1820),MorphDrone
2,258765,2,"[(15E0) , TrainProbe]",(15E0),TrainProbe
3,134801,2,"[(13E0) , TrainMarine]",(13E0),TrainMarine
4,90700,2,"[(1822) , MorphOverlord]",(1822),MorphOverlord
5,83416,2,"[(1821) , MorphZergling]",(1821),MorphZergling
6,51130,2,"[(1E60) , TrainQueen]",(1E60),TrainQueen
7,42761,2,"[(13A0) , LowerSupplyDepot]",(13A0),LowerSupplyDepot
8,34531,2,"[(1C60) , UpgradeToOrbitalCommand]",(1C60),UpgradeToOrbitalCommand
9,30424,2,"[(480) , Stop]",(480),Stop


In [30]:
# 셀렉션의 최빈값이 NULL 이다 = 무슨의미 일까? --- 모르면 feature로 넣어보고 평가!
counts['Selection'][:50]

[]                              811772
['OrbitalCommand [3100001]']     75165
['Nexus [3100001]']              55956
['OrbitalCommand [3440001]']     52412
['Lair [3100001]']               47356
['Nexus [3440001]']              38817
['Lair [3440001]']               29997
['Hive [3100001]']               28367
['SCV [37C0001]']                27178
['OrbitalCommand [3080001]']     23561
['OrbitalCommand [3540001]']     22337
['Hive [3440001]']               21290
['OrbitalCommand [3380001]']     19858
['OrbitalCommand [3000001]']     19846
['Barracks [39C0001]']           19781
['Hatchery [3100001]']           19022
['Nexus [3080001]']              17860
['SCV [38C0001]']                16964
['Barracks [3A40001]']           16768
['SCV [3740001]']                16366
['Barracks [3A00001]']           16349
['SCV [3580001]']                15993
['OrbitalCommand [33C0001]']     15658
['Lair [3080001]']               15492
['SCV [3780001]']                15384
['OrbitalCommand [2DC0001

## 공격,생산을 코드별로 분류해서 카운트 한다
- 질문 : 공격코드가 많을까? 생산코드가 많을까? 추측 = 진화코드가 많을까?
- 예) 5A0 = Attack; Target = Object; Location=좌표 ---> 공격 코드
- 단일코드) 1360 = TrainSCV / 1820=MorphDrone / 15E0 = TrainProbe = 생산유닛 뽑기를 하나로 묶음!


In [23]:
counts['Ability']

(1360) - TrainSCV                                                     350376
(1820) - MorphDrone                                                   287254
(15E0) - TrainProbe                                                   258765
(13E0) - TrainMarine                                                  134801
(1822) - MorphOverlord                                                 90700
                                                                       ...  
(1023) - BuildBarracks; Location: (67.5, 23.5, 40931)                      1
(5A0) - Attack; Location: (39.466796875, 130.158203125, 49136)             1
(5A0) - Attack; Location: (51.98828125, 127.69384765625, 32743)            1
(1025) - BuildMissileTurret; Location: (144.0, 21.0, 49132)                1
(5A0) - Attack; Location: (30.04833984375, 139.90283203125, 49136)         1
Name: event_contents, Length: 906693, dtype: int64

In [24]:
counts['Ability'][:60]

(1360) - TrainSCV                          350376
(1820) - MorphDrone                        287254
(15E0) - TrainProbe                        258765
(13E0) - TrainMarine                       134801
(1822) - MorphOverlord                      90700
(1821) - MorphZergling                      83416
(1E60) - TrainQueen                         51130
(13A0) - LowerSupplyDepot                   42761
(1C60) - UpgradeToOrbitalCommand            34531
(480) - Stop                                30424
(4C2) - HoldPosition                        24828
(1401) - BuildSiegeTank                     19363
(13E1) - TrainReaper                        19301
(1829) - MorphRoach                         18200
(1C80) - TransformToWarpGate                16109
(1420) - TrainMedivac                       15937
(1586) - TrainAdept                         15566
(1405) - BuildHellion                       15001
(17C1) - EvolveMetabolicBoost               14638
(31A0) - MedivacSpeedBoost                  14504


In [25]:
counts['Ability'][-60:-1]

(5A0) - Attack; Location: (113.933349609375, 41.077880859375, 49136)                                             1
(5A0) - Attack; Location: (18.4716796875, 155.790283203125, 49127)                                               1
(1543) - BuildGateway; Location: (135.5, 141.5, 49119)                                                           1
(1AC6) - TrainAdept; Location: (142.102294921875, 129.039306640625, 40931)                                       1
(5A0) - Attack; Target: SCV [03080001]; Location: (75.72509765625, 23.255615234375, 40928)                       1
(4E40) - KD8Charge; Location: (116.479736328125, 149.00146484375, 49105)                                         1
(2120) - BuildCreepTumor; Location: (87.5, 36.5, 32731)                                                          1
(5A0) - Attack; Location: (107.74853515625, 27.771728515625, 40931)                                              1
(5A0) - Attack; Location: (150.327392578125, 119.540771484375, 40926)           

# Null 타입 이벤트 (공격이벤트)
1. Ability
> - 어택 횟수가 공격에 미치는 영향
> - 훈련 횟수가 공격에 미치는 영향

1. 공격 이벤트 = Null 값 
> - 어느 이벤트가 더 영향을 미칠까? 모르겠으면 다 합한다.
> - 부대 지정횟수
> - 부대 추가횟수
> - 부대 불러오기 횟수
> - 부대 행동 횟수

1. Selection = Null 값 빈도가 제일 높음?
> - Null 값의 의미가 뭘까? 모르면 플레이어별로 카운트
> - Null 값이 + or - 영향인지 모름


| 컬럼명  | 값    | type             | 설명           | 대표값 |   비고 |
|:-------:|:------|------------------|----------------|:-------|:------:|
| event   |Ability           |Object |생산, 공격 등 선수의 주요 행동 | (code) | 어택, 훈련횟수 추출!
|         |Camera            |Object |시점 선택               | at(coords)  | 첫시점=시작포인트!
|         |SetControlGroup   |Object |부대 지정       |       NULL        
|         |AddToControlGroup |Object |부대에 추가     |       NULL        
|         |GetControlGroup   |Object |부대 불러오기   |       NULL        
|         |ControlGroup      |Object |부대 행동       |       NULL        
|         |Right             |Object |Click : 마우스 우클릭   | Target:(object); Location: at(coords)
|         |Selection         |Object |객체 선택       |NULL 값 추출 | 종류 너무 많음! 

In [26]:
counts['SetControlGroup']

Series([], Name: event_contents, dtype: int64)

In [27]:
counts['GetControlGroup']

Series([], Name: event_contents, dtype: int64)

In [28]:
counts['AddToControlGroup']

Series([], Name: event_contents, dtype: int64)

In [29]:
counts['ControlGroup']

Series([], Name: event_contents, dtype: int64)